In [1]:
import config as cfg
from test import test
from train import train
from utils import kfold
from models.efficientdeeplab import Backbones



for (train_patients, val_patients, test_patients) in kfold(cfg.DATASET_PATH):
    print(len(train_patients), len(val_patients), len(test_patients))
    # model, checkpoint_path = train(train_patients, val_patients, Backbones.B0, i = i)
    # test(model, test_patients, checkpoint_path, i = i)
    # break

2024-05-03 17:11:06.884368: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-03 17:11:07.627959: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64:
2024-05-03 17:11:07.628013: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64:
2024-05-03 17:11:07.628020: W tensorflow/compiler/tf2tensorrt/utils/py_uti

Segmentation Models: using `keras` framework.


2024-05-03 17:11:08.447043: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-03 17:11:08.452997: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-03 17:11:08.453201: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-03 17:11:08.453605: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

26 7 7
27 6 7
27 6 7


In [ ]:
# ===== TEST ===== #
from generators import PatientWiseGenerator
import tensorflow as tf
import numpy as np
from postprocessing import biggest_3D_object
import utils

model.load_weights(f"{checkpoint_path}/{model.name}")

metrics = {
    "dice":[],
    "precision":[],
    "recall":[],
    "iou_score":[],
}

metrics_post_processed = {
    "dice":[],
    "precision":[],
    "recall":[],
    "iou_score":[],
}

for patient in test_patients:
    generator = PatientWiseGenerator(patient, cfg.N_CLASSES)
    patient_dataset = tf.data.Dataset.from_generator(
        generator,
        output_signature=
        (
            tf.TensorSpec(shape=cfg.IMAGE_SHAPE + (cfg.N_CHANNELS,)),
            tf.TensorSpec(shape=cfg.IMAGE_SHAPE + (cfg.N_CLASSES,))
        )
    )
    
    volume_true = []
    for (x, y) in patient_dataset:
        volume_true.append(y)
    volume_true = np.squeeze(np.array(volume_true))

    volume_pred = model.predict(patient_dataset.batch(5), verbose=0)
    volume_pred = np.round(volume_pred)
    volume_pred = np.squeeze(volume_pred)
    
    metrics["dice"].append(cfg.f_score(volume_true, volume_pred))
    metrics["precision"].append(cfg.precision_score(volume_true, volume_pred))
    metrics["recall"].append(cfg.recall_score(volume_true, volume_pred))
    metrics["iou_score"].append(cfg.iou_score(volume_true, volume_pred))

    # post process each channel
    volume_post_processed = biggest_3D_object(volume_pred)
    
    metrics_post_processed["dice"].append(cfg.f_score(volume_true, volume_post_processed))
    metrics_post_processed["precision"].append(cfg.precision_score(volume_true, volume_post_processed))
    metrics_post_processed["recall"].append(cfg.recall_score(volume_true, volume_post_processed))
    metrics_post_processed["iou_score"].append(cfg.iou_score(volume_true, volume_post_processed))

output_folder = f"output/{model.name}_{i}"
utils.store_test_metrics(metrics, output_folder, "test_metrics")
utils.store_test_metrics(metrics_post_processed, output_folder, "test_metrics_post_processing")